# Implementing GAN Hacks to Train Stable Models

## 1. Deep Convolutional GANs(DCGANs)

### 1.1 Downsample Using Strided Convolutions
* The effect is the model will downsample the input from 64 x 64 to 32 x 32

In [1]:
# example of downsampling with strided convolutions
from keras.models import Sequential
from keras.layers import Conv2D
# define model
model = Sequential()
model.add(Conv2D(64,(3,3),strides=(2,2),padding='same',input_shape=(64,64,3)))
# summarize model
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 64)        1792      
Total params: 1,792
Trainable params: 1,792
Non-trainable params: 0
_________________________________________________________________


### 1.2 Upsample Using Strided Convolutions
* The effect is the model will upsample the input from 64 x 64 to 128 x 128.

In [2]:
# example of upsampling with strided convolutions
from keras.models import Sequential
from keras.layers import Conv2DTranspose
# define model
model = Sequential()
model.add(Conv2DTranspose(64,(4,4),strides=(2,2),padding='same',input_shape=(64,64,3)))
# summarize model
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_transpose (Conv2DTran (None, 128, 128, 64)      3136      
Total params: 3,136
Trainable params: 3,136
Non-trainable params: 0
_________________________________________________________________


### 1.3 Using Leaky ReLU
* Using the LeakyReLU with the default slope of 0.2 after a convolutional layer in a discriminator model.

In [3]:
# example of using leakyrelu in a discriminator model
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import LeakyReLU
# define model
model = Sequential()
model.add(Conv2D(64,(3,3),strides=(2,2),padding='same',input_shape=(64,64,3)))
model.add(LeakyReLU(0.2))
# summarize model
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 64)        1792      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 32, 32, 64)        0         
Total params: 1,792
Trainable params: 1,792
Non-trainable params: 0
_________________________________________________________________


### 1.4 Using Batch Normalization
* Batch Normalization standardizes the activations from a prior layer to have a zero mean and unit variance.

In [4]:
# example of using batch norm in a discriminator model
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
# define model
model = Sequential()
model.add(Conv2D(64,(3,3),strides=(2,2),padding='same',input_shape=(64,64,3)))
model.add(BatchNormalization())
model.add(LeakyReLU(0.2))
# summarize model
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 32, 32, 64)        1792      
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 64)        256       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32, 32, 64)        0         
Total params: 2,048
Trainable params: 1,920
Non-trainable params: 128
_________________________________________________________________


### 1.5 Using Gaussian Weight Initialization
* Initializing all weights using a zero-centered Gaussian distribution with the standard deviation of 0.02.

In [6]:
# example of gaussian weight initialization in a generator model
from keras.models import Sequential
from keras.layers import Conv2DTranspose
from keras.initializers import RandomNormal
# define model
model = Sequential()
init = RandomNormal(mean=0.0,stddev=0.02)
model.add(Conv2DTranspose(64,(4,4),strides=(2,2),padding='same',kernel_initializer=init,input_shape=(64,64,3)))
# summarize model
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_transpose_2 (Conv2DTr (None, 128, 128, 64)      3136      
Total params: 3,136
Trainable params: 3,136
Non-trainable params: 0
_________________________________________________________________


### 1.6 Using Adam Stochastic Gradient Descent
* Using the Adam version of stochastic gradient descent with the learning rate lr of 0.0002 and beta1 momentum value of 0.5.

In [8]:
# example of using adam when training a discriminator model
from keras.models import Sequential
from keras.layers import Conv2D
from keras.optimizers import Adam
# define model
model = Sequential()
model.add(Conv2D(64,(3,3),strides=(2,2),padding='same',input_shape=(64,64,3)))
# compile model
opt = Adam(lr=0.0002,beta_1=0.5)
model.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy'])
# summarize model
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 32, 32, 64)        1792      
Total params: 1,792
Trainable params: 1,792
Non-trainable params: 0
_________________________________________________________________


### 1.7 Scale Images to the Range[-1,1]
* Scaling a NumPy array of loaded image data to the required range of [-1,1].

In [9]:
# example of a function for scaling images
from numpy.random import randint

# scale image data from [0,255] to [-1,1]
def scale_images(images):
    # convert from unit8 to float32
    images = images.astype('float32')
    # scale from [0,255] to [-1,1]
    images = (images - 127.5)/127.5
    return images

# define one 28x28 color image
images = randint(0,256,28*28*3)
images = images.reshape((1,28,28,3))
# summarize pixel values
print(images.min(),images.max())
# scale
scaled = scale_images(images)
# summarize pixel scaled values
print(scaled.min(),scaled.max())

0 255
-1.0 1.0


## 2. Soumith Chintala's GAN Hacks

### 2.1 Using a Gaussian Latent Space
* The latent space defines the shape and distribution of the input to the generator model used to generate new images.

In [10]:
# example of sampling from a gaussian latent space
from numpy.random import randn

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim,n_samples):
    # generate points in the latent space
    x_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape((n_samples,latent_dim))
    return x_input

# size of latent space
n_dim = 100
# number of samples to generate
n_samples = 500
# generate samples
samples = generate_latent_points(n_dim,n_samples)
# summarize
print(samples.shape,samples.mean(),samples.std())

(500, 100) -0.003627560248747541 0.9997837729083461


### 2.2 Using Label Smoothing

In [1]:
# example of positive label smoothing
from numpy import ones
from numpy.random import random

# example of smoothing class=1 to [0.7,1.2]
def smooth_positive_labels(y):
    return y - 0.3 + (random(y.shape) * 0.5)

# generate 'real' class labels (1)
n_samples = 1000
y = ones((n_samples,1))
# smooth labels
y = smooth_positive_labels(y)
# summarize smooth labels
print(y.shape,y.min(),y.max())

(1000, 1) 0.7000677288314175 1.1996506274030443


In [3]:
# example of negative label smoothing
from numpy import zeros
from numpy.random import random

# example of smoothing class=0 to [0.0,0.3]
def smooth_negative_labels(y):
    return y + random(y.shape) * 0.3

# generate 'fake' class labels (0)
n_samples = 1000
y = zeros((n_samples,1))
# smooth labels
y = smooth_negative_labels(y)
# summarize smooth labels
print(y.shape,y.min(),y.max())

(1000, 1) 0.0002911431007373011 0.29899450217592727


### 2.3 Using Noisy Labels
*  It is to introduce some errors to these labels where some fake images are marked as real, and some real images are marked as fake.

In [5]:
# example of noisy labels
from numpy import ones
from numpy import zeros
from numpy.random import choice

# randomly flip some labels
def noisy_labels(y,p_flip):
    # determine the number of labels to flip
    n_select = int(p_flip * y.shape[0])
    # choose labels to flip
    flip_ix = choice([i for i in range(y.shape[0])],size=n_select)
    # invert the labels in place
    y[flip_ix] = 1 - y[flip_ix]
    return y

# generate 'real' class labels(1)
n_samples = 1000
y = ones((n_samples,1))
# flip labels with 5% probability
y = noisy_labels(y,0.05)
# summarize labels
print(y.sum())

# generate 'fake' class labels(0)
y = zeros((n_samples,1))
# flip labels with 5% probability
y = noisy_labels(y,0.05)
# summarize labels
print(y.sum())

950.0
49.0
